# 2. Get the Anecdote to Estimate Impact

Notebook to build all of the different stix objects associated with asking the reporting user about the Impact, and recording it through an Anecdote

We take v3.0 of the [Incident Core Extension](https://github.com/os-threat/cti-stix-common-objects/blob/main/extension-definition-specifications/incident-core/Incident%20Extension%20Suite.adoc) specification, with particular focus on the Sighting Extensions for Evidence, and combine it with the standard Stix approach. There are nine stages:

1. First setup the global parameters and retrieve the context memory
2. Create the Anecdote object and connect it to the Identity,
2. Add these objects to the `ObservedData` object
4. Create the `Sighting` object with the `SightingAnecdote` extension
5. Create the Impact object
5. Create the next Task and its Sequence objects
6. Connect the Sequence object to the previous sequence object
6. Add all of the objects to the existing Incident object
7. Save the context memory


Each time a block makes an object, and saves it as a json, this notebook will parse the object into an actual Stix object, so it can be bundled and printy printed (This step also verifies the objects are created correctly, and is cool).



## A. Load Imports

### A.1 StixORM Imports


In [12]:
import sys
!{sys.executable} -m pip install stixorm
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)


[notice] A new release of pip is available: 24.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### A.2 Relative Path Imports
https://stackoverflow.com/questions/76162459/jupyter-notebook-importerror-attempted-relative-import-with-no-known-parent-pac/77528726#77528726

In [13]:
import sys
sys.path.append('../')
import os

### A.3 Relative Import of Blocks

In [14]:
import json
from Block_Families.StixORM.SCO.Email_Message.make_email_msg import main as make_email_msg
from Block_Families.StixORM.SRO.Relationship.make_sro import main as make_sro
from Utilities.local_make_general import invoke_save_company_context_block, invoke_save_incident_context_block, invoke_get_from_company_block, invoke_get_from_user_block, invoke_move_unattached_to_other_block
from Utilities.local_make_sro import invoke_sro_block, invoke_sighting_block
from Utilities.local_make_sdo import (
    invoke_make_observed_data_block, invoke_make_indicator_block, invoke_make_event_block, invoke_make_sequence_block,
    invoke_make_task_block, invoke_make_incident_block, invoke_make_impact_block
)
from Utilities.local_make_sco import (
    invoke_make_email_addr_block, invoke_make_url_block, invoke_make_e_msg_block, invoke_make_anecdote_block
)
from conv import conv


context_base = "../Orchestration/generated/os-triage/context_mem/"
path_base = "../Block_Families/Objects/"
results_base = "../Orchestration/Results/"
sro_data = "SRO/Relationship/sro_derived.json"
alert_data = {
    "from": "evil@northkorea.nk",
    "to": "naive@example.com",
    "reporter": "Naive Smith",
    "url": "https://www.northkorea.nk/we/are/mad/"
}
alert_details = {
    "alert_name": "Suspcious Email",
    "alert_source": "user-report",
    "alert_log": "I have found a suspicious email"
}
anecdote_data = {
    "to": "naive@example.com"
}

TR_Context_Memory_Path = "./Context_Mem/Type_Refinery_Context.json"

### A.4 Retrieve the TR User Account, the Email Address, and the TR Identity from the Context Memory

and the previous task sequence

In [15]:
# 1. For Naive Smith, find the user account and the email. Plus get my own identity
reporter_name = alert_data["reporter"]
laptop_name = "HP Laptop 1"
TR_name = "Trusty Jones"
#
# 2. Setup variables and queries to get the objects from options
# get "HP Laptop 1"
context_type = {
    "context_type": "assets"
}
laptop_ident_query = {
    "type" : "identity",
    "property": {
        "path": ["name"],
        "source_value": laptop_name,
        "comparator": "EQ"
    }
}
laptop_identity = invoke_get_from_company_block(laptop_ident_query, context_type, source_value=None, source_id=None)
# get reporter
context_type = {
    "context_type": "users"
}
reporter_ident_query = {
    "type" : "identity",
    "property": {
        "path": ["name"],
        "source_value": reporter_name,
        "comparator": "EQ"
    }
}
reporter_identity = invoke_get_from_company_block(reporter_ident_query, context_type, source_value=None, source_id=None)
context_type = {
    "context_type": "me"
}
TR_ident_query = {
    "type" : "identity",
    "property": {
        "path": ["name"],
        "source_value": TR_name,
        "comparator": "EQ"
    }
}
TR_identity = invoke_get_from_user_block(TR_ident_query, context_type, source_value=None, source_id=None)
#
print(f"\nreporter_identity-->{reporter_identity}")
print(f"\nlaptop_identity-->{laptop_identity}")
print(f"\nlaptop_identity-->{laptop_identity}")
print(f"\nTR_identity-->{TR_identity}")
#
# 3. save the objects to "unattached"
#
context_type = {
    "context_type": "unattached"
}
ident_results_obj_path = results_base + "/step0/Usr1__ident.json"
ident_results_context_path = results_base  + "/step1/context/usr1__ident_context.json"
result = invoke_save_incident_context_block(ident_results_obj_path, ident_results_context_path, context_type)
print(f" result->{result}")
ident_results_obj_path = results_base  + "/step0/TR_Identity__ident.json"
ident_results_context_path = results_base + "/step1/context/TR_Identity__ident_context.json"
result = invoke_save_incident_context_block(ident_results_obj_path, ident_results_context_path, context_type)
print(f" result->{result}")
ident_results_obj_path = results_base  + "/step0/Systems_Laptop1__ident.json"
ident_results_context_path = results_base + "/step1/context/Systems_Laptop1__ident_context.json"
result = invoke_save_incident_context_block(ident_results_obj_path, ident_results_context_path, context_type)
print(f" result->{result}")

company query->{'type': 'identity', 'property': {'path': ['name'], 'source_value': 'HP Laptop 1', 'comparator': 'EQ'}}
company query->{'type': 'identity', 'property': {'path': ['name'], 'source_value': 'Naive Smith', 'comparator': 'EQ'}}

reporter_identity-->{'type': 'identity', 'spec_version': '2.1', 'id': 'identity--090c9c16-cbbb-49e2-9289-bc1ad4cd7f77', 'created': '2025-07-09T21:43:55.998Z', 'modified': '2025-07-09T21:43:55.998Z', 'name': 'Naive Smith', 'description': 'A Naive Individual', 'roles': ['user', 'sales'], 'identity_class': 'individual', 'sectors': ['technology'], 'extensions': {'extension-definition--66e2492a-bbd3-4be6-88f5-cc91a017a498': {'extension_type': 'property-extension', 'contact_numbers': [{'contact_number_type': 'work-phone', 'contact_number': '0499-999-109'}], 'email_addresses': [{'digital_contact_type': 'work', 'email_address_ref': 'email-addr--4722424c-7012-56b0-84d5-01d076fc547b'}], 'first_name': 'Naive', 'last_name': 'Smith', 'middle_name': 'Weakling', 'pr

## B Make the Anecdote

In [16]:
# 1. Setup the form paths and the results path
anecdote_form = "SCO/Anecdote/anecdote_on_impact.json"
results_path = "step2/step2"
# 2. Invoke the Make Anecdote block
anecdote = invoke_make_anecdote_block(anecdote_form, results_path, anecdote_reporter=reporter_identity)
# 3. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "unattached"
}
anecdote_results_obj_path = results_base + results_path + "__anecdote.json"
anecdote_results_context_path = results_base + "/step2/anecdote_context.json"
result = invoke_save_incident_context_block(anecdote_results_obj_path, anecdote_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "anecdote",
    "spec_version": "2.1",
    "id": "anecdote--e1298bc0-818e-5cdb-9154-eac37c8e260f",
    "value": "I clicked on the link, and my laptop screen went weird",
    "report_date": "2025-07-09T21:47:49.468383Z",
    "provided_by_ref": "identity--090c9c16-cbbb-49e2-9289-bc1ad4cd7f77"
}
input data->{'stix_object': {'type': 'anecdote', 'spec_version': '2.1', 'id': 'anecdote--e1298bc0-818e-5cdb-9154-eac37c8e260f', 'value': 'I clicked on the link, and my laptop screen went weird', 'report_date': '2025-07-09T21:47:49.468383Z', 'provided_by_ref': 'identity--090c9c16-cbbb-49e2-9289-bc1ad4cd7f77'}, 'context_type': {'context_type': 'unattached'}}
from ports 
stix_object->{'type': 'anecdote', 'spec_version': '2.1', 'id': 'anecdote--e1298bc0-818e-5cdb-9154-eac37c8e260f', 'value': 'I clicked on the link, and my laptop screen went weird', 'report_date': '2025-07-09T21:47:49.468383Z', 'provided_by_ref': 'identity--090c9c16-cbbb-49e2-9289-bc1ad4cd7f77'}
context type->unattached
 

## C. Collect the Elements of the Observation into an Observed-Data Object

Tbis then represents the potential phishing email as a group of elements

In [17]:
# 1. Setup0 list of obsrvations
observed_ids = []
observed_ids.append(anecdote)
# 2. Setup path to form and results
obs_path ="SDO/Observed_Data/observation-alert.json"
results_path ="step2/observation-anecdote.json"
# 3. Invoke the Make Observed Data Block
obs_2 = invoke_make_observed_data_block(obs_path, results_path, observation=observed_ids)
# 4. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "unattached"
}
observation_results_obj_path = results_base + "/step2/observation-anecdote.json"
observation_results_context_path = results_base + "/step2/observation_anecdote_context.json"
result = invoke_save_incident_context_block(observation_results_obj_path, observation_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "observed-data",
    "spec_version": "2.1",
    "id": "observed-data--a3c5080d-f832-42f0-b9cd-e0dc8e4ec8fb",
    "created": "2025-07-09T21:47:49.599Z",
    "modified": "2025-07-09T21:47:49.599Z",
    "first_observed": "2020-10-19T01:01:01Z",
    "last_observed": "2020-10-19T01:01:01Z",
    "number_observed": 1,
    "object_refs": [
        "anecdote--e1298bc0-818e-5cdb-9154-eac37c8e260f"
    ]
}
input data->{'stix_object': {'type': 'observed-data', 'spec_version': '2.1', 'id': 'observed-data--a3c5080d-f832-42f0-b9cd-e0dc8e4ec8fb', 'created': '2025-07-09T21:47:49.599Z', 'modified': '2025-07-09T21:47:49.599Z', 'first_observed': '2020-10-19T01:01:01.000Z', 'last_observed': '2020-10-19T01:01:01.000Z', 'number_observed': 1, 'object_refs': ['anecdote--e1298bc0-818e-5cdb-9154-eac37c8e260f']}, 'context_type': {'context_type': 'unattached'}}
from ports 
stix_object->{'type': 'observed-data', 'spec_version': '2.1', 'id': 'observed-data--a3c5080d-f832-42f0-b9cd-e0dc8e4ec8fb', 'creat

## D. Create the Anecdote Sighting

Tbis then represents the Anecdote on Impact from the reporting user

### D.1 Create the Sighting
Connecting the:
- observed-data object
- the reporter identity as the sighted object
- the laptop identity as the location

In [18]:
# 1. Setup path to form and results
sighting_data_path ="SRO/Sighting/sighting_anecdote.json"
results_path ="step2/sighting_anecdote.json"
# 2. Setup the SDO sighted, the Observed-Data that was observed with generated objects,
#                        then the identity object from the context storage (note the slight difference in indexing #
observed_list = [obs_2]
sighted = reporter_identity
TR_where_list = [TR_identity]
# 2. Invoke the Make Observed Data Block
sight2 = invoke_sighting_block(sighting_data_path, results_path, observed=observed_list, sighted=sighted, where=TR_where_list)
# 3. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "unattached"
}
sighting_results_obj_path = results_base + "/step2/sighting_anecdote.json"
sighting_results_context_path = results_base + "/step2/sighting_anecdote_context.json"
result = invoke_save_incident_context_block(sighting_results_obj_path, sighting_results_context_path, context_type)
print(f" result->{result}")

<class 'dict'>
['sighting_form', 'observed_data_refs', 'where_sighted_refs', 'sighting_of_ref']
{
    "type": "sighting",
    "spec_version": "2.1",
    "id": "sighting--a58eb679-1677-47b2-8e55-4892ea57e996",
    "created": "2025-07-09T21:47:49.807Z",
    "modified": "2025-07-09T21:47:49.807Z",
    "count": 1,
    "sighting_of_ref": "identity--090c9c16-cbbb-49e2-9289-bc1ad4cd7f77",
    "observed_data_refs": [
        "observed-data--a3c5080d-f832-42f0-b9cd-e0dc8e4ec8fb"
    ],
    "where_sighted_refs": [
        "identity--37154aac-c730-4782-9144-3f0f9f9d7c95"
    ],
    "extensions": {
        "extension-definition--0d76d6d9-16ca-43fd-bd41-4f800ba8fc43": {
            "extension_type": "property-extension"
        },
        "sighting-anecdote": {
            "person_name": "user-report",
            "person_context": "employee",
            "report_submission": "interview"
        }
    }
}
input data->{'stix_object': {'type': 'sighting', 'spec_version': '2.1', 'id': 'sighting--a58eb

### D.2 Move Unattached to Other

In [19]:
total_list = observed_ids
total_list.append(obs_2)
total_list.append(TR_identity)
total_list.append(sight2)
total_list.append(reporter_identity)

obs_context_move_path = results_base + "step2/context/context_move.json"
obs_context_move_results = results_base + "step2/context/context_move_results.json"
result = invoke_move_unattached_to_other_block(obs_context_move_path, obs_context_move_results, total_list)

## E. Create the Impact Objects

Tbis Impact represents the effect reported by the user in the Anecdote

In [20]:
# 1. Setup path to form and results
impact_path ="SDO/Impact/anecdote_impact.json"
results_path ="step2/impact_anecdote.json"
# 2. Setup the number of assets impacted
numbers = {"computers-mobile": 1}
impacted_refs = [laptop_identity["id"]]
# 2. Invoke the Make Observed Data Block
impact_1 = invoke_make_impact_block(impact_path, results_path, impacted_entity_counts=numbers, impacted_refs=impacted_refs, superseded_by_ref=None)
# 3. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "impact"
}
impact_results_obj_path = results_base + results_path
impact_results_context_path = results_base + "/step2/impact_anecdote_context.json"
result = invoke_save_incident_context_block(impact_results_obj_path, impact_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "impact",
    "spec_version": "2.1",
    "id": "impact--98ce7265-4229-4114-b51e-c7e5550f7ffd",
    "created": "2025-07-09T21:47:50.528Z",
    "modified": "2025-07-09T21:47:50.528Z",
    "impact_category": "availability",
    "criticality": 99,
    "description": "Laptop is stuffed",
    "end_time": "2025-07-09T21:47:50.528Z",
    "impacted_entity_counts": {
        "computers-mobile": 1
    },
    "impacted_refs": [
        "identity--38647650-4e31-4c84-af51-4d97bb1a38a6"
    ],
    "recoverability": "regular",
    "start_time": "2025-07-09T21:47:50.528Z",
    "extensions": {
        "availability": {
            "availability_impact": 99
        },
        "extension-definition--7cc33dd6-f6a1-489b-98ea-522d351d71b9": {
            "extension_type": "new-sdo"
        }
    }
}
input data->{'stix_object': {'type': 'impact', 'spec_version': '2.1', 'id': 'impact--98ce7265-4229-4114-b51e-c7e5550f7ffd', 'created': '2025-07-09T21:47:50.528Z', 'modified': '2025-07-09T21:47:50.52

## F. Create the Next Task Object

Next step is to investigate the Exchange server to see who else got the email

### F.1 Create the Task Object

In [21]:
# 1. Setup path to form and results
task_data_path ="SDO/Task/task_anecdote.json"
results_path ="step2/task_anecdote.json"
# 2. Invoke the Make Observed Data Block
task_2 = invoke_make_task_block(task_data_path, results_path, changed_objects=None)
# 3. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "task"
}
task_results_obj_path = results_base + "/step2/task_anecdote.json"
task_results_context_path = results_base + "/step2/task_anecdote_context.json"
result = invoke_save_incident_context_block(task_results_obj_path, task_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "task",
    "spec_version": "2.1",
    "id": "task--3aa55e73-842b-4b2f-b3d7-9beaed047844",
    "created": "2025-07-09T21:47:50.755Z",
    "modified": "2025-07-09T21:47:50.755Z",
    "description": "Find out who else got the email",
    "end_time": "2025-07-09T21:47:50.755Z",
    "name": "Query Exchange Server",
    "start_time": "2025-07-09T21:47:50.755Z",
    "extensions": {
        "extension-definition--2074a052-8be4-4932-849e-f5e7798e0030": {
            "extension_type": "new-sdo"
        }
    }
}
input data->{'stix_object': {'type': 'task', 'spec_version': '2.1', 'id': 'task--3aa55e73-842b-4b2f-b3d7-9beaed047844', 'created': '2025-07-09T21:47:50.755Z', 'modified': '2025-07-09T21:47:50.755Z', 'description': 'Find out who else got the email', 'end_time': '2025-07-09T21:47:50.755Z', 'name': 'Query Exchange Server', 'start_time': '2025-07-09T21:47:50.755Z', 'extensions': {'extension-definition--2074a052-8be4-4932-849e-f5e7798e0030': {'extension_type': 'new-sdo'}}}, 'co

### F.2 Create the Sequence Object

In [22]:
# 1. Setup path to form and results
sequence_data_path ="SDO/Sequence/sequence_anecdote.json"
results_path ="step2/sequence_task_anecdote.json"
# 2. Setup the Sequence Object for the Event
#
step_type = "single_step"
sequence_type = "task"
sequenced_object = task_2["id"]
# 2. Invoke the Make Observed Data Block
seq_T_2 = invoke_make_sequence_block(sequence_data_path, results_path, step_type=step_type, sequence_type=sequence_type, sequenced_object=sequenced_object, on_completion=None, on_success=None, on_failure=None, next_steps=None)
# 3. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "sequence"
}
sequence_results_obj_path = results_base + "/step2/sequence_task_anecdote.json"
sequence_results_context_path = results_base + "/step2/sequence_anecdote_context.json"
result = invoke_save_incident_context_block(sequence_results_obj_path, sequence_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "sequence",
    "spec_version": "2.1",
    "id": "sequence--175c176f-f662-4622-b0c1-f2b84f111bd7",
    "created": "2025-07-09T21:47:50.957Z",
    "modified": "2025-07-09T21:47:50.957Z",
    "sequenced_object": "task--3aa55e73-842b-4b2f-b3d7-9beaed047844",
    "sequence_type": "task",
    "step_type": "single_step",
    "extensions": {
        "extension-definition--be0c7c79-1961-43db-afde-637066a87a64": {
            "extension_type": "new-sdo"
        }
    }
}
input data->{'stix_object': {'type': 'sequence', 'spec_version': '2.1', 'id': 'sequence--175c176f-f662-4622-b0c1-f2b84f111bd7', 'created': '2025-07-09T21:47:50.957Z', 'modified': '2025-07-09T21:47:50.957Z', 'sequenced_object': 'task--3aa55e73-842b-4b2f-b3d7-9beaed047844', 'sequence_type': 'task', 'step_type': 'single_step', 'extensions': {'extension-definition--be0c7c79-1961-43db-afde-637066a87a64': {'extension_type': 'new-sdo'}}}, 'context_type': {'context_type': 'sequence'}}
from ports 
stix_object->{'type': 'se

### F.3 Connect the Sequence to the Previous one